In [1]:
import argparse
import csv
import json
import random

import lexicon
import config
import utils
import torch
import math

from collections import defaultdict
from semantic_memory import taxonomy, vsm, vsm_utils
from nltk.corpus import wordnet as wn
from ordered_set import OrderedSet

In [2]:
# def get_dupes(L):
#     seen = set()
#     seen2 = set()
#     seen_add = seen.add
#     seen2_add = seen2.add
#     for item in L:
#         if item in seen:
#             seen2_add(item)
#         else:
#             seen_add(item)
#     return list(seen2)

# glove_vocab = []
# with open("../../glove.vocab") as f:
#     for line in f:
#         glove_vocab.append(line.strip("\n"))

# # len(set(glove_vocab))
# # get_dupes(glove_vocab)
# glove.embeddings.shape, len(set(glove_vocab))

# glove = vsm.VectorSpaceModel("glove", 300)
# glove.load_vectors("../../glove.840B.300d.txt")

# len(glove.vocab2idx)


# glove_queries = []
# glove_reduced_vocab = []
# all_concepts = concept_universe.union(set(anchor_children.keys()))
# for c in all_concepts:
#     if c in glove.vocab:
#         # pass
#         glove_queries.append(glove(c))
#         glove_reduced_vocab.append(c)
#     else:
#         try:
#             glove_queries.append(glove(c.split(" ")).mean(0).reshape(1, -1))
#             glove_reduced_vocab.append(c)
#         except:
#             print(c)
#             pass

# glove_queries = torch.stack(glove_queries, 1).squeeze(0)

# len(glove_queries), len(glove_reduced_vocab)

# glove_reduced = vsm.VectorSpaceModel("glove_queries")
# glove_reduced.load_vectors_from_tensor(glove_queries, glove_reduced_vocab)

In [3]:
VECTORS="../../lmms-sp-wsd.albert-xxlarge-v2.synsets.vectors.txt"

sense_embeddings = vsm.VectorSpaceModel("LMMS-ALBERT")
# takes about a min to load...
sense_embeddings.load_vectors(VECTORS)

117659it [00:56, 2080.35it/s]


In [235]:
sense_embeddings.neighbor("tamale.n.01", k=20)

[('tabora.n.01', 0.7659215927124023),
 ('hargeisa.n.01', 0.7564389705657959),
 ('cameroon.n.01', 0.7562030553817749),
 ('gafsa.n.01', 0.7523672580718994),
 ('huainaputina.n.01', 0.7497690916061401),
 ('mbeya.n.01', 0.749377965927124),
 ('kananga.n.01', 0.7490277290344238),
 ('sousse.n.01', 0.7465986013412476),
 ('cakchiquel.n.01', 0.7465184926986694),
 ('kota.n.01', 0.7464815974235535),
 ('grand_canal.n.02', 0.7463891506195068),
 ('kekchi.n.01', 0.745223879814148),
 ('aerie.n.02', 0.7440038323402405),
 ("akwa'ala.n.01", 0.7439688444137573),
 ('tarabulus.n.01', 0.7434621453285217),
 ('dobrich.n.01', 0.7422606945037842),
 ('homyel.n.01', 0.7406438589096069),
 ('mandara.n.01', 0.740449070930481),
 ('torreon.n.01', 0.7402486205101013),
 ('mukalla.n.01', 0.739410400390625)]

In [254]:
def lemma2concept(entry):
    return lexicon.Concept(
        lemma=entry["lemma"],
        singular=entry["singular"],
        plural=entry["plural"],
        article=entry["article"],
        generic=entry["generic"],
        taxonomic_phrase=entry["taxonomic_phrase"],
    )

In [272]:
lemma_path = "../data/things/things-lemmas-annotated.csv"

# triples_prompts = []

# read in concepts
concepts = defaultdict(lexicon.Concept)
with open(lemma_path, "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        # concepts.append(lemma2concept(row))
        if row["remove"] != "1":
            concepts[row["lemma"]] = lemma2concept(row)

In [273]:
# concepts['school bus']

In [306]:
things_senses = set()
concepts_annotated_senses = defaultdict(set)
with open("../data/things/things-senses-annotated.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row["sense"] != "-":
            senses = row['sense'].split("&")
            for sense in senses:
                things_senses.add(sense)
                concepts_annotated_senses[row["concept"]].add(sense)

concepts_annotated_senses = dict(concepts_annotated_senses)
# concepts_annotated_senses = 

In [307]:
for concept, senses in concepts_annotated_senses.items():
    for s in senses:
        try:
            sense_embeddings(s)
        except:
            print(concept, s)

women's clothing womens_clothing.n.00
school supply school_supply.n.00
breakfast breakfast.n.00
office supply office_supply.n.00


In [308]:
triple_path = "../data/things/things-triples.csv"
triples = utils.read_csv_dict(triple_path)

In [309]:
concept_universe = set()
anchor_synsets = defaultdict(str)
anchor_children = defaultdict(set)
synset_anchors = defaultdict(str)
nonsense = set()
hypernyms = defaultdict(str)
for triple in triples:
    hypernym = triple["hypernym"]
    hyponym = triple["hyponym"]
    anchor = triple["anchor"]
    if triple["anchor-sense"] == "-":
        nonsense.add(anchor)
        if anchor in [
            "breakfast",
            "office supply",
            "school supply",
            "women's clothing",
        ]:
            if anchor == "breakfast":
                anchor_sense = "breakfast.n.00"
            elif anchor == "office supply":
                anchor_sense = "office_supply.n.00"
            elif anchor == "school supply":
                anchor_sense = "school_supply.n.00"
            elif anchor == "women's clothing":
                anchor_sense = "womens_clothing.n.00"
        anchor_synsets[anchor] = anchor_sense
        synset_anchors[anchor_sense] = anchor
        hypernyms[hyponym] = anchor
    else:
        anchor_sense = triple["anchor-sense"]
        if anchor == "headwear":
            # print(triple)
            anchor_sense = "headdress.n.01"
        elif anchor == "toy":
            anchor_sense = "plaything.n.01"
        elif anchor == "protective clothing":
            anchor_sense = "protective_covering.n.01"
        elif anchor == "breakfast":
            anchor_sense = "breakfast.n.00"
        elif anchor == "office supply":
            anchor_sense = "office_supply.n.00"
        elif anchor == "school supply":
            anchor_sense = "school_supply.n.00"
        elif anchor == "women's clothing":
            anchor_sense = "womens_clothing.n.00"
        if anchor_sense == "none":
            print(triple)
        anchor_synsets[anchor] = anchor_sense
        synset_anchors[anchor_sense] = anchor
        hypernyms[hyponym] = anchor
    # concept_universe.add(hypernym)
    concept_universe.add(hyponym)
    # concept_universe.add(anchor)
    anchor_children[anchor].add(hyponym)

anchor_children = dict(anchor_children)
anchor_synsets = dict(anchor_synsets)
synset_anchors = dict(synset_anchors)
hypernyms = dict(hypernyms)
nonsense

{'breakfast', 'office supply', 'school supply', "women's clothing"}

In [310]:
NONSENSES = {
    "breakfast.n.00": [
        "bacon.n.01",
        "bagel.n.01",
        "bread.n.01",
        "breakfast.n.01",
        "breakfast_food.n.01",
        "cereal.n.03",
        "cream_cheese.n.01",
        "crepe.n.01",
        "crescent_roll.n.01",
        "doughnut.n.02",
        "egg.n.02",
        "french_fries.n.01",
        "granola.n.01",
        "grits.n.01",
        "ham.n.01",
        "hamburger.n.01",
        "hash.n.01",
        "honey.n.01",
        "jam.n.01",
        "maple_syrup.n.01",
        "marmalade.n.01",
        "muffin.n.01",
        "oatmeal.n.01",
        "omelet.n.01",
        "pancake.n.01",
        "pastry.n.02",
        "peanut_butter.n.01",
        "sandwich.n.01",
        "sausage.n.01",
        "scone.n.01",
        "scrambled_eggs.n.01",
        "syrup.n.01",
        "toast.n.01",
        "waffle.n.01",
        "yogurt.n.01",
    ],
    "office_supply.n.00": [
        "binder.n.03",
        "clipboard.n.01",
        "envelope.n.01",
        "eraser.n.01",
        "file.n.01",
        "booklet.n.01",
        "fountain_pen.n.01",
        "glue.n.01",
        "highlighter.n.01",
        "ink.n.01",
        "notebook.n.01",
        "notepad.n.01",
        "paper.n.01",
        "paper_clip.n.01",
        "pen.n.01",
        "pencil.n.01",
        "pencil_sharpener.n.01",
        "punch.n.03",
        "rubber_band.n.01",
        "scissors.n.01",
        "staple.n.05",
        "stapler.n.01",
        "tack.n.02",
        "tape.n.01",
        "thumbtack.n.01",
    ],
    "school_supply.n.00": [
        "backpack.n.01",
        "binder.n.03",
        "book.n.01",
        "calculator.n.02",
        "chalk.n.04",
        "blackboard.n.01",
        "crayon.n.01",
        "eraser.n.01",
        "booklet.n.01",
        "fountain_pen.n.01",
        "glue.n.01",
        "highlighter.n.01",
        "ink.n.01",
        "notebook.n.01",
        "inkwell.n.01",
        "marker.n.03",
        "notepad.n.01",
        "paper.n.01",
        "paper_clip.n.01",
        "pen.n.01",
        "pencil.n.01",
        "pencil_sharpener.n.01",
        "scissors.n.01",
        "staple.n.05",
        "stapler.n.01",
    ],
    "womens_clothing.n.00": [
        "bikini.n.02",
        "blouse.n.01",
        "feather_boa.n.01",
        "brassiere.n.01",
        "corset.n.01",
        "dress.n.01",
        "nylons.n.01",
        "garter.n.01",
        "headscarf.n.01",
        "kimono.n.01",
        "legging.n.01",
        "lingerie.n.01",
        "pants_suit.n.01",
        "pantyhose.n.01",
        "skirt.n.02",
        "stocking.n.01",
        "tiara.n.01",
        "head_covering.n.01",
    ],
}

In [311]:
for a in anchor_synsets:
    if a == 'none':
        print(a)

In [312]:
# things_senses

In [313]:
# space = []
# for concept_universe in concept_universe - anchor_children['animal']:


def synset_names(concept, return_map=False, both=False):
    # synset_list = [synset.name() for synset in wn.synsets(concept, "n")]
    synset_list = [s for s in concepts_annotated_senses[concept]]
    if return_map:
        synset_map = {s: concept for s in synset_list}
        concept_map = {concept: s for s in synset_list}
        return synset_map, concept_map
    elif both:
        return (
            synset_list,
            {s: concept for s in synset_list},
            {concept: s for s in synset_list},
        )
    else:
        return synset_list

In [314]:
# synset_names("tupperware")
# sense_embeddings("bread-bin.n.01")

In [315]:
synset_map = {}
concept_map = {}
concept_synsets = defaultdict(set)
for concept in concept_universe:
    lst, smap, cmap = synset_names(concept, both=True)
    synset_map.update(smap)
    concept_map.update(cmap)
    concept_synsets[concept].update(set(lst))
    # synset_map.update(synset_names(concept, return_map=True))

concept_synsets = dict(concept_synsets)

In [316]:
def is_hypernym(synset, target):
    try:
        target = wn.synset(target)
        for path in wn.synset(synset).hypernym_paths():
            if target in path:
                return True
    except:
        return False
    return False

In [317]:
all_synsets = set()
for k,v in concept_synsets.items():
    for vv in v:
        if vv in things_senses:
            all_synsets.add(vv)
    # all_synsets.update(v)

In [318]:
synset_universe = set(list(all_synsets) + list(anchor_synsets.values()))
synset_universe = [s for s in synset_universe if s not in NONSENSES.keys()]

In [319]:
len(synset_universe)

1508

In [320]:
# "school_bus.n.01" in all_synsets
# sense_embeddings("school_bus.n.01")

# sense_embeddings("stirfry.n.01")
for w in sense_embeddings.vocab:
    if "substance" in w:
        print(w)



liquid_body_substance.n.01
substance.n.01
transparent_substance.n.01
humic_substance.n.01
body_substance.n.01
substance_abuse.n.01
controlled_substance.n.01
substance.n.07
solid_body_substance.n.01
substance.n.04


In [321]:
# sense_embeddings(synset_universe)
universe_vectors = sense_embeddings(synset_universe)

excess = []
excess_vocab = []
for k, v in NONSENSES.items():
    excess.append(sense_embeddings(v).mean(0))
    excess_vocab.append(k)

reduced_sense_embeddings = vsm.VectorSpaceModel("Reduced Sense Embs")
reduced_sense_embeddings.load_vectors_from_tensor(
    torch.cat((universe_vectors, torch.stack(excess))), synset_universe + excess_vocab
)

reduced_sense_embeddings

<Reduced Sense Embs VectorSpaceModel: 1512 x 4096>

In [322]:
reduced_sense_embeddings.neighbor("office_supply.n.00",  k = 10)

[('school_supply.n.00', 0.9904826879501343),
 ('highlighter.n.02', 0.9274128675460815),
 ('latch.n.02', 0.9172189235687256),
 ('dustpan.n.02', 0.9132122993469238),
 ('backscratcher.n.02', 0.9129491448402405),
 ('bolt.n.06', 0.912151575088501),
 ('whisk.n.01', 0.9098642468452454),
 ('stretcher.n.03', 0.9062033891677856),
 ('envelope.n.02', 0.9012345671653748),
 ('top.n.10', 0.9010876417160034)]

In [20]:
anchor_neighbors = defaultdict(list)
for anchor, anchor_synset in anchor_synsets.items():
    # leftover = set()
    # anchor = synset_anchors[anchor_synset]
    space = concept_universe - anchor_children[anchor]
    space_synsets = set()
    for c in space:
        if c in concepts.keys():
            for cs in concept_synsets[c]:
                if cs not in anchor_synsets.values() and cs in synset_universe:
                    space_synsets.add(cs)

    space_synsets = list(space_synsets)
    # filter out hypernymy cases
    space_synsets = [s for s in space_synsets if is_hypernym(s, anchor_synset) == False]
    # space_synsets_filtered = []
    # for s in space_synsets:
    #     if s in NONSENSES.keys():
    #         pass
    #     elif is_hypernym(s, anchor_synset) == False:
    #         space_synsets_filtered.append(s)

    neighbors = reduced_sense_embeddings.neighbor(
        anchor_synset,
        space=space_synsets,
        k=len(anchor_children[anchor]),
        names_only=True,
        ignore_first=False,
    )[0]
    neighbor_concepts = [synset_map[n] for n in neighbors]
    neighbor_concepts = list(OrderedSet(neighbor_concepts))[
        : math.ceil(len(anchor_children[anchor]) / 2)
    ]
    anchor_neighbors[anchor].extend(neighbor_concepts)

    # non-neighbors
    non_neighbors = reduced_sense_embeddings.neighbor(
        anchor_synset,
        space=space_synsets,
        k=len(anchor_children[anchor]),
        names_only=True,
        ignore_first=False,
        nearest=False,
    )[0]
    non_neighbor_concepts = [synset_map[n] for n in non_neighbors]
    non_neighbor_concepts = list(OrderedSet(non_neighbor_concepts))[
        : math.ceil(len(anchor_children[anchor]) / 2)
    ]
    anchor_neighbors[anchor].extend(non_neighbor_concepts)

anchor_neighbors = dict(anchor_neighbors)
random.seed(42)
anchor_neighbors = {k: random.sample(v, len(v)) for k, v in anchor_neighbors.items()}

In [21]:
# check if any negatively sampled concept is a children (shouldn't be)
for anchor, neighbors in anchor_neighbors.items():
    for n in neighbors:
        if n in anchor_children[anchor]:
            print(anchor, n)

In [22]:
# just generate the negatives sample triples
44 * 1307

57508

In [23]:
len(concept_universe)

1300

In [24]:
sum([len(v) for k,v in anchor_children.items()])

2032

In [25]:
sum([len(v) for k,v in anchor_neighbors.items()])

2054

In [26]:
anchor_synsets['animal']

'animal.n.01'

In [27]:
negative_sample_triples = []
for anchor, negative_samples in anchor_neighbors.items():
    for ns in negative_samples:
        negative_sample_triples.append((anchor, anchor_synsets[anchor], ns))

In [28]:
len(negative_sample_triples)

2054

In [29]:
# concepts['garden tool']
# concepts['crib']
'crib' in concepts.keys()

False

In [30]:
with open("../data/things/things-sense_based_ns-triples.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["anchor", "anchor-sense", "hyponym"])
    for triple in negative_sample_triples:
        writer.writerow(triple)

In [31]:
synset_map['bird.n.01']

'bird'

In [32]:
# save anchor, every concept similarities
anchor_concept_sims = defaultdict(list)
for anchor, anchor_synset in anchor_synsets.items():
    space = concept_universe
    space_synsets = set()
    for c in space:
        if c in concepts.keys():
            for cs in concept_synsets[c]:
                if cs not in anchor_synsets.values() and cs in synset_universe:
                    space_synsets.add(cs)
    neighbor_sims = reduced_sense_embeddings.neighbor(anchor_synset, k = len(space_synsets), space=space_synsets, ignore_first=False)
    for concept_sense, sim in neighbor_sims[0]:
        try:
            concept = synset_map[concept_sense]
        except:
            synset_anchors[concept_sense]
        anchor_concept_sims[(anchor, concept)].append(sim)

anchor_concept_sims_csv = []
for (anchor, concept), sims in anchor_concept_sims.items():
    anchor_concept_sims_csv.append((anchor, concept, max(sims)))

In [33]:
len(anchor_concept_sims_csv)

45496

In [34]:
with open("../data/things/things-sense_based_anchor_sims.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["anchor", "concept", "similarity"])
    for row in anchor_concept_sims_csv:
        writer.writerow(row)

In [63]:
for anchor, synset in anchor_synsets.items():
    if len(anchor_children[anchor]) != len(anchor_neighbors[anchor]):
        print(anchor, len(anchor_children[anchor]), len(anchor_neighbors[anchor]))

home decor 45 46
insect 17 18
plant 47 48
animal 177 178
toiletry 31 32
container 105 106
kitchen tool 27 28
musical instrument 33 34
breakfast 35 36
headwear 19 20
medical equipment 27 28
accessory 37 38
condiment 15 16
jewelry 15 16
footwear 15 16
office supply 25 26
drink 19 20
water vehicle 19 20
bird 27 28
game 19 20
dessert 37 38
garden tool 17 18
